1. Write a decorator that ensures a function is only called by users with a
 specific role. Each function should have an user_type with a string 
type in kwargs. Example:

@is_admin
def show_customer_receipt(user_type: str):
    # Some very dangerous operation

show_customer_receipt(user_type='user')
> ValueError: Permission denied

show_customer_receipt(user_type='admin')
> function pass as it should be

In [3]:
from functools import wraps

def is_admin(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        user_type = kwargs.get('user_type')
        if user_type != 'admin':
            raise ValueError('Permission denied')
        return func(*args, **kwargs)
    return wrapper

@is_admin
def show_customer_receipt(user_type: str):
    # Some very dangerous operation
    print("Customer receipt shown successfully")

show_customer_receipt(user_type='user')  # Raises ValueError: Permission denied

show_customer_receipt(user_type='admin')  # Prints "Customer receipt shown successfully"


ValueError: Permission denied

2. Write a decorator that wraps a function in a try-except block and prints an error if an error has happened. Example:

@catch_errors
def some_function_with_risky_operation(data):
    print(data['key'])


some_function_with_risky_operation({'foo': 'bar'})
> Found 1 error during execution of your function: KeyError no such key as foo

some_function_with_risky_operation({'key': 'bar'})
> bar



In [4]:
def catch_errors(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Found 1 error during execution of your function: {type(e).__name__} {str(e)}")
    return wrapper

@catch_errors
def some_function_with_risky_operation(data):
    print(data['key'])

some_function_with_risky_operation({'foo': 'bar'})  # Prints "Found 1 error during execution of your function: KeyError 'key'"
some_function_with_risky_operation({'key': 'bar'})  # Prints "bar"

Found 1 error during execution of your function: KeyError 'key'
bar


3. Optional: Create a decorator that will check types. It should take a function with arguments and validate inputs with annotations. It should work for all possible functions. Don`t forget to check the return type as well
Example:
@check_types
def add(a: int, b: int) -> int:
    return a + b

add(1, 2)
> 3

add("1", "2")
> TypeError: Argument a must be int, not str


In [5]:
import inspect

def check_types(func):
    sig = inspect.signature(func)
    
    def check_arguments(*args, **kwargs):
        bound_args = sig.bind(*args, **kwargs)
        for name, value in bound_args.arguments.items():
            param = sig.parameters[name]
            expected_type = param.annotation
            if expected_type != inspect.Parameter.empty and not isinstance(value, expected_type):
                raise TypeError(f"Argument {name} must be {expected_type.__name__}, not {type(value).__name__}")
        return func(*args, **kwargs)
    
    def check_return(*args, **kwargs):
        result = check_arguments(*args, **kwargs)
        expected_return_type = sig.return_annotation
        if expected_return_type != inspect.Parameter.empty and not isinstance(result, expected_return_type):
            raise TypeError(f"Return value must be {expected_return_type.__name__}, not {type(result).__name__}")
        return result
    
    return check_return


In [6]:
@check_types
def add(a: int, b: int) -> int:
    return a + b

add(1, 2)  # Returns 3

add("1", "2")  # Raises TypeError: Argument a must be int, not str

TypeError: Argument a must be int, not str

4. Optional: Create a function that caches the result of a function, so that if it is called with the same argument multiple times, it returns the cached result first instead of re-executing the function.

In [7]:
from functools import wraps

def memoize(func):
    cache = {}
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        key = (args, tuple(kwargs.items()))
        if key in cache:
            return cache[key]
        result = func(*args, **kwargs)
        cache[key] = result
        return result
    
    return wrapper

In [8]:
@memoize
def expensive_function(n):
    print(f"Calculating result for {n}...")
    return n * 2

print(expensive_function(5))  # Calculates the result and returns 10
print(expensive_function(5))  # Returns the cached result of 10
print(expensive_function(10))  # Calculates the result and returns 20
print(expensive_function(10))  # Returns the cached result of 20


Calculating result for 5...
10
10
Calculating result for 10...
20
20


5. Optional: Write a decorator that adds a rate-limiter to a function, so that it can only be called a certain amount of times per minute

In [9]:
import time
from functools import wraps

def rate_limiter(limit, per_seconds):
    interval = 1 / per_seconds
    call_times = []

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()

            # Remove old timestamps that are outside the time frame
            call_times[:] = [t for t in call_times if now - t <= limit]

            if len(call_times) >= limit:
                # Rate limit exceeded
                raise ValueError(f"Rate limit exceeded. Only {limit} calls allowed per {limit * interval} seconds.")

            call_times.append(now)
            return func(*args, **kwargs)

        return wrapper

    return decorator


In [10]:
@rate_limiter(limit=3, per_seconds=60)
def my_function():
    print("Function called")

my_function()  # Prints "Function called"
my_function()  # Prints "Function called"
my_function()  # Prints "Function called"
my_function()  # Raises ValueError: Rate limit exceeded. Only 3 calls allowed per 60.0 seconds.


Function called
Function called
Function called


ValueError: Rate limit exceeded. Only 3 calls allowed per 0.05 seconds.